In [1]:
# Import required libraries
from azure.ai.ml import MLClient

ModuleNotFoundError: No module named 'azure.ai'

In [2]:
pip uninstall azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --pre azure-ai-ml

     |████████████████████████████████| 2.7 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 137 kB 40.6 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 44.8 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 366 kB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 35.8 MB/s eta 0:00:01
  Created wheel for strictyaml: filename=strictyaml-1.6.1-py3-none-any.whl size=123915 sha256=dc933b2321b2cd090d55ff75c68f44c0369708356a2b035e460fc1699e826762
  Stored in directory: /home/azureuser/.cache/pip/wheels/24/8b/79/d11bb76d63bc1b37df7bd35dc1e47577eb926b7eb1b619fc2d
Successfully built strictyaml
ERROR: fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
ERROR: pycaret 2.3.10 has requirement pyyaml<6.0.0, but you'll have pyya

In [1]:
# Import required libraries
from azure.ai.ml import MLClient

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [3]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<<RESOURCE_GROUP_NAME>>",
        "resource_group": "<<RESOURCE_GROUP_NAME>>",
        "workspace_name": "<<MACHINE_LEARNING_WORKSPACE_NAME>>",
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: ../.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fc7e9976b50>,
         subscription_id=<<SUBSCRIPTION_ID>>,
         resource_group_name=<<RESOURCE_GROUP_NAME>>,
         workspace_name=<<MACHINE_LEARNING_WORKSPACE_NAME>>)


In [4]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "ComputeClusterDemo"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Compute cluster not found...")
    raise ex